In [1]:
import cobra
import cobra.test
import os
import numpy as np
from copy import copy
import re
import pandas as pd
import sys
#sys.path.insert(0, "../../segre/cometspy") # works now with installed 0.4.16
import cometspy as c

import itertools
os.environ["GUROBI_COMETS_HOME"] = "/opt/gurobi900/linux64"


## figure out the parameters that reduce the affected's growth by 50% in glucose
Use the new multitoxin format

In [2]:

unlimited = ["EX_ca2_e","EX_cl_e","EX_co2_e","EX_cobalt2_e","EX_cu2_e",
             "EX_fe2_e","EX_fe3_e","EX_h_e","EX_h2o_e","EX_k_e","EX_mg2_e",
             "EX_mn2_e","EX_mobd_e","EX_na1_e","EX_nh4_e","EX_ni2_e",
             "EX_o2_e","EX_pi_e","EX_sel_e","EX_slnt_e","EX_so4_e",
             "EX_tungs_e","EX_zn2_e"]


model_dir = "../kinkel_smanski_data/ten_genomes_from_stephen_feb_2020"

# 4 chosen strains: S3211.3 (3B),  S3212.3 (3F), S3212.4 (3G), S3213.3 (3I).
model_ids = ["ST32113_grampos",   #3B
             "ST32123_grampos",   #3F
             "ST32124_grampos",   #3G
             "ST32133_grampos"]  #3I

name_dict = {"ST32113_grampos": "B",
             "ST32123_grampos": "F",
             "ST32124_grampos": "G",
             "ST32133_grampos": "I"}

combos = list(itertools.combinations(model_ids, 4))

In [3]:
cobra_models = []
for model_id in combos[0]:
    model, warnings = cobra.io.validate_sbml_model(f"{model_dir}/{model_id}_2022gapclosed.xml")
    cobra_models.append(model)
B = [m for m in cobra_models if m.id == "ST32113_grampos_2022gapclosed"][0]
F = [m for m in cobra_models if m.id == "ST32123_grampos_2022gapclosed"][0]
G = [m for m in cobra_models if m.id == "ST32124_grampos_2022gapclosed"][0]
I = [m for m in cobra_models if m.id == "ST32133_grampos_2022gapclosed"][0]

Academic license - for non-commercial use only - expires 2022-07-22
Using license file /home/jeremy/gurobi.lic


SBML errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.
SBML errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.
SBML errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.
SBML errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.


In [4]:

### Make the cobra models which make toxin, export it, and see it
from cobra import Metabolite, Reaction

B_toxin_c = Metabolite("B_toxin_c", compartment = "C_c")
B_toxin_e = Metabolite("B_toxin_e", compartment = "C_e")
B_objective = B.reactions.Growth
# arbitrarily have one unit of toxin produced for 1 gram of cells
B_objective.add_metabolites({B_toxin_c: 1.})
B_toxin_tpp = Reaction("B_toxin_tpp", 
                       lower_bound = 0, upper_bound = 1000)
B_toxin_tpp.add_metabolites({B_toxin_c : -1,
                       B_toxin_e: 1})
B.add_reactions([B_toxin_tpp])
B.add_boundary(B_toxin_e, type = "exchange",
               lb = 0, ub = 1000)

B.reactions.EX_glc__D_e.lower_bound = -10

F_toxin_c = Metabolite("F_toxin_c", compartment = "C_c")
F_toxin_e = Metabolite("F_toxin_e", compartment = "C_e")
F_objective = F.reactions.Growth
# arbitrarily have one unit of toxin produced for 1 gram of cells
F_objective.add_metabolites({F_toxin_c: 1.})
F_toxin_tpp = Reaction("F_toxin_tpp", 
                       lower_bound = 0, upper_bound = 1000)
F_toxin_tpp.add_metabolites({F_toxin_c : -1,
                       F_toxin_e: 1})
F.add_reactions([F_toxin_tpp])
F.add_boundary(F_toxin_e, type = "exchange",
               lb = 0, ub = 1000)

F.reactions.EX_glc__D_e.lower_bound = -10

G_toxin_c = Metabolite("G_toxin_c", compartment = "C_c")
G_toxin_e = Metabolite("G_toxin_e", compartment = "C_e")
G_objective = G.reactions.Growth
# arbitrarily have one unit of toxin produced for 1 gram of cells
G_objective.add_metabolites({G_toxin_c: 1.})
G_toxin_tpp = Reaction("G_toxin_tpp", 
                       lower_bound = 0, upper_bound = 1000)
G_toxin_tpp.add_metabolites({G_toxin_c : -1,
                       G_toxin_e: 1})
G.add_reactions([G_toxin_tpp])
G.add_boundary(G_toxin_e, type = "exchange",
               lb = 0, ub = 1000)

G.reactions.EX_glc__D_e.lower_bound = -10

# B needs to see F,G
# F needs to see B,G
#G needs to see B
#I needs to see B

B.add_boundary(F_toxin_e, type = "exchange",
               lb = 0, ub = 0)
B.add_boundary(G_toxin_e, type = "exchange",
               lb = 0, ub = 0)
F.add_boundary(B_toxin_e, type = "exchange",
               lb = 0, ub = 0)
F.add_boundary(G_toxin_e, type = "exchange",
               lb = 0, ub = 0)
G.add_boundary(B_toxin_e, type = "exchange",
               lb = 0, ub = 0)
I.add_boundary(B_toxin_e, type = "exchange",
               lb = 0, ub = 0)


Reaction identifier,EX_B_toxin_e
Name,exchange
Memory address,0x07f159fa45550
Stoichiometry,B_toxin_e --> -->
GPR,
Lower bound,0
Upper bound,0


In [5]:

### Make the COMETS models 
B_comets = c.model(B)
B_comets.id = "B"
F_comets = c.model(F)
F_comets.id = "F"
G_comets = c.model(G)
G_comets.id = "G"
I_comets = c.model(I)
I_comets.id = "I"

B_comets.open_exchanges()
F_comets.open_exchanges()
G_comets.open_exchanges()
I_comets.open_exchanges()

B_comets.initial_pop = [[0,0, 0.01]]
F_comets.initial_pop = [[0,0, 0.01]]
G_comets.initial_pop = [[0,0, 0.01]]
I_comets.initial_pop = [[0,0, 0.01]]


# First effect of F on B

In [6]:
models = [B_comets, F_comets]
params = c.params()
params.set_param("timeStep", 1/60.)
params.set_param("maxCycles", 800)
params.set_param("spaceWidth", 10) # cm
params.set_param("defaultKm", 0.000001)
params.set_param("maxSpaceBiomass", 1000)

layout = c.layout(models)
for met in unlimited:
    met = met[3:]
    if met in layout.media.metabolite.values:
        layout.set_specific_metabolite(met, 1000.)
layout.set_specific_metabolite("glc__D_e", 5.)

sim = c.comets(layout, params)
sim.classpath_pieces['bin'] = 'home/jeremy/Dropbox/work_related/harcombe_lab/segre/comets_multitoxin.jar'
sim.working_dir = "../comets_tests/"
sim.run(delete_files = True)
control = sim.total_biomass


Running COMETS simulation ...
Done!


In [7]:
control.loc[control.cycle == 800, "B"]

800    0.237116
Name: B, dtype: float64

## do an initial sweep of 3 parameter values, then grid search within that

In [8]:
# B get the toxins. a multitoxin effect of F and G
B.medium = {key: 10 for key in unlimited if key in [ex.id for ex in B.exchanges]}
B.reactions.EX_glc__D_e.lower_bound = -10.
B_gr_max = B.slim_optimize()
print(f"B_gr_max: {B_gr_max}")


results = []
kms = [1.e-5, 1.e-4, 1.e-3]
for km in kms:
    B_comets = c.model(B)
    B_comets.id = "B"
    B_comets.open_exchanges()
    B_comets.initial_pop = [[0,0, 0.01]]
    rxn_num = 2138 # obj
    exch_inds = [229] # f toxin, g toxin
    bound = "ub"
    vmax = B_gr_max
    hills = [5]
    B_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [B_comets, F_comets]
    params = c.params()
    params.set_param("timeStep", 1/60.)
    params.set_param("maxCycles", 800)
    params.set_param("spaceWidth", 10) # cm
    params.set_param("defaultKm", 0.000001)
    params.set_param("maxSpaceBiomass", 1000)

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "B"].values[0] / control.loc[control.cycle == 800, "B"].values[0]
    results.append(proportion)

print(results)
results_backup = results

B_gr_max: 0.6938262254347477

Running COMETS simulation ...
Done!

Running COMETS simulation ...


FileNotFoundError: [Errno 2] No such file or directory: '../comets_tests/B.cmd'

In [90]:
results = results_backup
kms = [1.e-5, 1.e-4, 1.e-3]
results

[0.08594080916503756, 0.47297561330009735, 1.0]

In [92]:
# now loop through new midpoints until we get close to 50% growth
while any(np.abs(np.array(results) - 0.5) > 0.0001):
    print(f"current kms: {kms}")
    print(f"current proportions: {np.array(results)}")
    start = max([i for i in range(len(results)) if results[i] < 0.5])
    end = min([i for i in range(len(results)) if results[i] > 0.5])
    first_km = kms[start]
    last_km = kms[end]
    new_middle = (first_km + last_km) / 2
    kms = [first_km, new_middle, last_km]
    results = [results[start],
                    0,
                    results[end]]
    km = new_middle
    
    B_comets = c.model(B)
    B_comets.id = "B"
    B_comets.open_exchanges()
    B_comets.initial_pop = [[0,0, 0.01]]
    B_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [B_comets, F_comets]

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "B"].values[0] / control.loc[control.cycle == 800, "B"].values[0]
    results[1] = proportion
                 

current kms: [0.00010351562500000001, 0.0001052734375, 0.00010703125]
current proportions: [0.49001514 0.49868166 0.50749216]

Running COMETS simulation ...
Done!
current kms: [0.0001052734375, 0.00010615234375, 0.00010703125]
current proportions: [0.49868166 0.5030461  0.50749216]

Running COMETS simulation ...
Done!
current kms: [0.0001052734375, 0.00010571289062500001, 0.00010615234375]
current proportions: [0.49868166 0.50085832 0.5030461 ]

Running COMETS simulation ...
Done!
current kms: [0.0001052734375, 0.00010549316406250001, 0.00010571289062500001]
current proportions: [0.49868166 0.49978695 0.50085832]

Running COMETS simulation ...
Done!
current kms: [0.00010549316406250001, 0.00010560302734375001, 0.00010571289062500001]
current proportions: [0.49978695 0.50034041 0.50085832]

Running COMETS simulation ...
Done!
current kms: [0.00010549316406250001, 0.00010554809570312502, 0.00010560302734375001]
current proportions: [0.49978695 0.50006361 0.50034041]

Running COMETS simul

# The final iteration showed that using the below kms for the effect of F on B resulted in the below reductions in final relative yield: 

current kms: [0.00010549316406250001, 0.00010552062988281251, 0.00010554809570312502]
current proportions: [0.49978695 0.49992527 0.50006361]

## Now do effect of G on B

Final stats:

current kms: [0.0001013029098510742, 0.00010131235122680663, 0.00010132179260253905]

current proportions: [0.49988324 0.49998617 0.5000888 ]

In [93]:
B_comets = c.model(B)
B_comets.id = "B"
B_comets.open_exchanges()
B_comets.initial_pop = [[0,0, 0.01]]
models = [B_comets, G_comets]
params = c.params()
params.set_param("timeStep", 1/60.)
params.set_param("maxCycles", 800)
params.set_param("spaceWidth", 10) # cm
params.set_param("defaultKm", 0.000001)
params.set_param("maxSpaceBiomass", 1000)

layout = c.layout(models)
for met in unlimited:
    met = met[3:]
    if met in layout.media.metabolite.values:
        layout.set_specific_metabolite(met, 1000.)
layout.set_specific_metabolite("glc__D_e", 5.)

sim = c.comets(layout, params)
sim.classpath_pieces['bin'] = 'home/jeremy/Dropbox/work_related/harcombe_lab/segre/comets_multitoxin.jar'
sim.working_dir = "../comets_tests/"
sim.run(delete_files = True)
control = sim.total_biomass

# B get the toxins. a multitoxin effect of F and G
B.medium = {key: 10 for key in unlimited if key in [ex.id for ex in B.exchanges]}
B.reactions.EX_glc__D_e.lower_bound = -10.
B_gr_max = B.slim_optimize()
print(f"B_gr_max: {B_gr_max}")


results = []
kms = [1.e-6, 1.e-4, 1.e-2]
for km in kms:
    B_comets = c.model(B)
    B_comets.id = "B"
    B_comets.open_exchanges()
    B_comets.initial_pop = [[0,0, 0.01]]
    rxn_num = 2138 # obj
    exch_inds = [230] # g toxin
    bound = "ub"
    vmax = B_gr_max
    hills = [5]
    B_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [B_comets, G_comets]
    params = c.params()
    params.set_param("timeStep", 1/60.)
    params.set_param("maxCycles", 800)
    params.set_param("spaceWidth", 10) # cm
    params.set_param("defaultKm", 0.000001)
    params.set_param("maxSpaceBiomass", 1000)

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "B"].values[0] / control.loc[control.cycle == 800, "B"].values[0]
    results.append(proportion)

print(results)

# now loop through new midpoints until we get close to 50% growth
while any(np.abs(np.array(results) - 0.5) > 0.0001):
    print(f"current kms: {kms}")
    print(f"current proportions: {np.array(results)}")
    start = max([i for i in range(len(results)) if results[i] < 0.5])
    end = min([i for i in range(len(results)) if results[i] > 0.5])
    first_km = kms[start]
    last_km = kms[end]
    new_middle = (first_km + last_km) / 2
    kms = [first_km, new_middle, last_km]
    results = [results[start],
                    0,
                    results[end]]
    km = new_middle
    
    B_comets = c.model(B)
    B_comets.id = "B"
    B_comets.open_exchanges()
    B_comets.initial_pop = [[0,0, 0.01]]
    B_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [B_comets, G_comets]

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "B"].values[0] / control.loc[control.cycle == 800, "B"].values[0]
    results[1] = proportion
                 


Running COMETS simulation ...
Done!
B_gr_max: 0.6938262254347549

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!
[0.03844564338617583, 0.48782692819390405, 0.9999999973041714]
current kms: [1e-06, 0.0001, 0.01]
current proportions: [0.03844564 0.48782693 1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.00505, 0.01]
current proportions: [0.48782693 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.002575, 0.00505]
current proportions: [0.48782693 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.0013375, 0.002575]
current proportions: [0.48782693 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.00071875, 0.0013375]
current proportions: [0.48782693 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.000409375, 0.00071875]
current proportions: [0.48782693 1.01634425 1. 

## G on F

current kms: [0.00010335994958877564, 0.00010336053967475891, 0.00010336112976074219]
current proportions: [0.49977176 0.50037698 0.50035437]

In [94]:
F_comets = c.model(F)
F_comets.id = "F"
F_comets.open_exchanges()
F_comets.initial_pop = [[0,0, 0.01]]
models = [F_comets, G_comets]
params = c.params()
params.set_param("timeStep", 1/60.)
params.set_param("maxCycles", 800)
params.set_param("spaceWidth", 10) # cm
params.set_param("defaultKm", 0.000001)
params.set_param("maxSpaceBiomass", 1000)

layout = c.layout(models)
for met in unlimited:
    met = met[3:]
    if met in layout.media.metabolite.values:
        layout.set_specific_metabolite(met, 1000.)
layout.set_specific_metabolite("glc__D_e", 5.)

sim = c.comets(layout, params)
sim.classpath_pieces['bin'] = 'home/jeremy/Dropbox/work_related/harcombe_lab/segre/comets_multitoxin.jar'
sim.working_dir = "../comets_tests/"
sim.run(delete_files = True)
control = sim.total_biomass

# F growth rate max
F.medium = {key: 10 for key in unlimited if key in [ex.id for ex in F.exchanges]}
F.reactions.EX_glc__D_e.lower_bound = -10.
F_gr_max = F.slim_optimize()
print(f"F_gr_max: {F_gr_max}")


results = []
kms = [1.e-6, 1.e-4, 1.e-2]
for km in kms:
    F_comets = c.model(F)
    F_comets.id = "F"
    F_comets.open_exchanges()
    F_comets.initial_pop = [[0,0, 0.01]]
    rxn_num = 2246 # obj
    exch_inds = [265] # g toxin
    bound = "ub"
    vmax = F_gr_max
    hills = [5]
    F_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [F_comets, G_comets]
    params = c.params()
    params.set_param("timeStep", 1/60.)
    params.set_param("maxCycles", 800)
    params.set_param("spaceWidth", 10) # cm
    params.set_param("defaultKm", 0.000001)
    params.set_param("maxSpaceBiomass", 1000)

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "F"].values[0] / control.loc[control.cycle == 800, "F"].values[0]
    results.append(proportion)

print(results)

# now loop through new midpoints until we get close to 50% growth
while any(np.abs(np.array(results) - 0.5) > 0.0001):
    print(f"current kms: {kms}")
    print(f"current proportions: {np.array(results)}")
    start = max([i for i in range(len(results)) if results[i] < 0.5])
    end = min([i for i in range(len(results)) if results[i] > 0.5])
    first_km = kms[start]
    last_km = kms[end]
    new_middle = (first_km + last_km) / 2
    kms = [first_km, new_middle, last_km]
    results = [results[start],
                    0,
                    results[end]]
    km = new_middle
    
    F_comets = c.model(F)
    F_comets.id = "F"
    F_comets.open_exchanges()
    F_comets.initial_pop = [[0,0, 0.01]]
    F_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [F_comets, G_comets]

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "F"].values[0] / control.loc[control.cycle == 800, "F"].values[0]
    results[1] = proportion
                 


Running COMETS simulation ...
Done!
F_gr_max: 0.6975020973020472

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!
[0.03858852134353666, 0.4738381133493845, 1.0]
current kms: [1e-06, 0.0001, 0.01]
current proportions: [0.03858852 0.47383811 1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.00505, 0.01]
current proportions: [0.47383811 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.002575, 0.00505]
current proportions: [0.47383811 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.0013375, 0.002575]
current proportions: [0.47383811 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.00071875, 0.0013375]
current proportions: [0.47383811 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.000409375, 0.00071875]
current proportions: [0.47383811 0.9949615  1.        ]

Runnin

KeyboardInterrupt: 

## B on F

kms = [0.00010309677124023437, 0.00010313453674316405, 0.00010317230224609375]

results = [0.49983510336563575, 0.5000249034669171, 0.5002154043665212]

In [95]:
F_comets = c.model(F)
F_comets.id = "F"
F_comets.open_exchanges()
F_comets.initial_pop = [[0,0, 0.01]]

B_comets = c.model(B)
B_comets.id = "B"
B_comets.open_exchanges()
B_comets.initial_pop = [[0,0, 0.01]]

models = [F_comets, B_comets]
params = c.params()
params.set_param("timeStep", 1/60.)
params.set_param("maxCycles", 800)
params.set_param("spaceWidth", 10) # cm
params.set_param("defaultKm", 0.000001)
params.set_param("maxSpaceBiomass", 1000)

layout = c.layout(models)
for met in unlimited:
    met = met[3:]
    if met in layout.media.metabolite.values:
        layout.set_specific_metabolite(met, 1000.)
layout.set_specific_metabolite("glc__D_e", 5.)

sim = c.comets(layout, params)
sim.classpath_pieces['bin'] = 'home/jeremy/Dropbox/work_related/harcombe_lab/segre/comets_multitoxin.jar'
sim.working_dir = "../comets_tests/"
sim.run(delete_files = True)
control = sim.total_biomass

# F growth rate max
F.medium = {key: 10 for key in unlimited if key in [ex.id for ex in F.exchanges]}
F.reactions.EX_glc__D_e.lower_bound = -10.
F_gr_max = F.slim_optimize()
print(f"F_gr_max: {F_gr_max}")


results = []
kms = [1.e-6, 1.e-4, 1.e-2]
for km in kms:
    F_comets = c.model(F)
    F_comets.id = "F"
    F_comets.open_exchanges()
    F_comets.initial_pop = [[0,0, 0.01]]
    rxn_num = 2246 # obj
    exch_inds = [264] # b toxin
    bound = "ub"
    vmax = F_gr_max
    hills = [5]
    F_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [F_comets, B_comets]
    params = c.params()
    params.set_param("timeStep", 1/60.)
    params.set_param("maxCycles", 800)
    params.set_param("spaceWidth", 10) # cm
    params.set_param("defaultKm", 0.000001)
    params.set_param("maxSpaceBiomass", 1000)

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "F"].values[0] / control.loc[control.cycle == 800, "F"].values[0]
    results.append(proportion)

print(results)

# now loop through new midpoints until we get close to 50% growth
while all(np.abs(np.array(results) - 0.5) > 0.0001):
    print(f"current kms: {kms}")
    print(f"current proportions: {np.array(results)}")
    start = max([i for i in range(len(results)) if results[i] < 0.5])
    end = min([i for i in range(len(results)) if results[i] > 0.5])
    first_km = kms[start]
    last_km = kms[end]
    new_middle = (first_km + last_km) / 2
    kms = [first_km, new_middle, last_km]
    results = [results[start],
                    0,
                    results[end]]
    km = new_middle
    
    F_comets = c.model(F)
    F_comets.id = "F"
    F_comets.open_exchanges()
    F_comets.initial_pop = [[0,0, 0.01]]
    F_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [F_comets, B_comets]

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "F"].values[0] / control.loc[control.cycle == 800, "F"].values[0]
    results[1] = proportion
                 


Running COMETS simulation ...
Done!
F_gr_max: 0.6975020973020472

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!
[0.04697269324951742, 0.4845074732101491, 1.0]
current kms: [1e-06, 0.0001, 0.01]
current proportions: [0.04697269 0.48450747 1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.00505, 0.01]
current proportions: [0.48450747 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.002575, 0.00505]
current proportions: [0.48450747 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.0013375, 0.002575]
current proportions: [0.48450747 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.00071875, 0.0013375]
current proportions: [0.48450747 1.         1.        ]

Running COMETS simulation ...
Done!
current kms: [0.0001, 0.000409375, 0.00071875]
current proportions: [0.48450747 0.99567388 1.        ]

Runnin

## B on G

kms = [8.083984375e-05, 8.088378906250001e-05, 8.0927734375e-05]

results = [0.49967355380150896, 0.4999108161416882, 0.5001480716406231]

In [98]:
G_comets = c.model(G)
G_comets.id = "G"
G_comets.open_exchanges()
G_comets.initial_pop = [[0,0, 0.01]]

B_comets = c.model(B)
B_comets.id = "B"
B_comets.open_exchanges()
B_comets.initial_pop = [[0,0, 0.01]]

models = [G_comets, B_comets]
params = c.params()
params.set_param("timeStep", 1/60.)
params.set_param("maxCycles", 800)
params.set_param("spaceWidth", 10) # cm
params.set_param("defaultKm", 0.000001)
params.set_param("maxSpaceBiomass", 1000)

layout = c.layout(models)
for met in unlimited:
    met = met[3:]
    if met in layout.media.metabolite.values:
        layout.set_specific_metabolite(met, 1000.)
layout.set_specific_metabolite("glc__D_e", 5.)

sim = c.comets(layout, params)
sim.classpath_pieces['bin'] = 'home/jeremy/Dropbox/work_related/harcombe_lab/segre/comets_multitoxin.jar'
sim.working_dir = "../comets_tests/"
sim.run(delete_files = True)
control = sim.total_biomass

# G growth rate max
G.medium = {key: 10 for key in unlimited if key in [ex.id for ex in G.exchanges]}
G.reactions.EX_glc__D_e.lower_bound = -10.
G_gr_max = G.slim_optimize()
print(f"G_gr_max: {G_gr_max}")


results = []
kms = [1.e-5, 1.e-4, 1.e-3]
for km in kms:
    G_comets = c.model(G)
    G_comets.id = "G"
    G_comets.open_exchanges()
    G_comets.initial_pop = [[0,0, 0.01]]
    rxn_num = 2139 # obj
    exch_inds = [234] # b toxin
    bound = "ub"
    vmax = G_gr_max
    hills = [5]
    G_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [G_comets, B_comets]
    params = c.params()
    params.set_param("timeStep", 1/60.)
    params.set_param("maxCycles", 800)
    params.set_param("spaceWidth", 10) # cm
    params.set_param("defaultKm", 0.000001)
    params.set_param("maxSpaceBiomass", 1000)

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "G"].values[0] / control.loc[control.cycle == 800, "G"].values[0]
    results.append(proportion)



# now loop through new midpoints until we get close to 50% growth
while all(np.abs(np.array(results) - 0.5) > 0.0001):
    print(f"current kms: {kms}")
    print(f"current proportions: {np.array(results)}")
    start = max([i for i in range(len(results)) if results[i] < 0.5])
    end = min([i for i in range(len(results)) if results[i] > 0.5])
    first_km = kms[start]
    last_km = kms[end]
    new_middle = (first_km + last_km) / 2
    kms = [first_km, new_middle, last_km]
    results = [results[start],
                    0,
                    results[end]]
    km = new_middle
    
    G_comets = c.model(G)
    G_comets.id = "G"
    G_comets.open_exchanges()
    G_comets.initial_pop = [[0,0, 0.01]]
    G_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [G_comets, B_comets]

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "G"].values[0] / control.loc[control.cycle == 800, "G"].values[0]
    results[1] = proportion
                 


Running COMETS simulation ...
Done!
F_gr_max: 0.6954727730133302

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!
current kms: [1e-05, 0.0001, 0.001]
current proportions: [0.1109411  0.60131278 1.        ]

Running COMETS simulation ...
Done!
current kms: [1e-05, 5.5e-05, 0.0001]
current proportions: [0.1109411  0.35872092 0.60131278]

Running COMETS simulation ...
Done!
current kms: [5.5e-05, 7.75e-05, 0.0001]
current proportions: [0.35872092 0.48162035 0.60131278]

Running COMETS simulation ...
Done!
current kms: [7.75e-05, 8.875e-05, 0.0001]
current proportions: [0.48162035 0.5422034  0.60131278]

Running COMETS simulation ...
Done!
current kms: [7.75e-05, 8.3125e-05, 8.875e-05]
current proportions: [0.48162035 0.51202994 0.5422034 ]

Running COMETS simulation ...
Done!
current kms: [7.75e-05, 8.031250000000001e-05, 8.3125e-05]
current proportions: [0.48162035 0.49682585 0.51202994]

Running COMETS simulation ...
Done!
c

## B on I

kms = [8.1279296875e-05, 8.13232421875e-05, 8.13671875e-05]

results [0.4995427253875728, 0.49997474416161036, 0.5003344988950044]


In [100]:
I_comets = c.model(I)
I_comets.id = "I"
I_comets.open_exchanges()
I_comets.initial_pop = [[0,0, 0.01]]

B_comets = c.model(B)
B_comets.id = "B"
B_comets.open_exchanges()
B_comets.initial_pop = [[0,0, 0.01]]

models = [I_comets, B_comets]
params = c.params()
params.set_param("timeStep", 1/60.)
params.set_param("maxCycles", 800)
params.set_param("spaceWidth", 10) # cm
params.set_param("defaultKm", 0.000001)
params.set_param("maxSpaceBiomass", 1000)

layout = c.layout(models)
for met in unlimited:
    met = met[3:]
    if met in layout.media.metabolite.values:
        layout.set_specific_metabolite(met, 1000.)
layout.set_specific_metabolite("glc__D_e", 5.)

sim = c.comets(layout, params)
sim.classpath_pieces['bin'] = 'home/jeremy/Dropbox/work_related/harcombe_lab/segre/comets_multitoxin.jar'
sim.working_dir = "../comets_tests/"
sim.run(delete_files = True)
control = sim.total_biomass

# I growth rate max
I.medium = {key: 10 for key in unlimited if key in [ex.id for ex in I.exchanges]}
I.reactions.EX_glc__D_e.lower_bound = -10.
I_gr_max = I.slim_optimize()
print(f"I_gr_max: {I_gr_max}")


results = []
kms = [1.e-5, 1.e-4, 1.e-3]
for km in kms:
    I_comets = c.model(I)
    I_comets.id = "I"
    I_comets.open_exchanges()
    I_comets.initial_pop = [[0,0, 0.01]]
    rxn_num = 2488 # obj
    exch_inds = [275] # b toxin
    bound = "ub"
    vmax = I_gr_max
    hills = [5]
    I_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [I_comets, B_comets]
    params = c.params()
    params.set_param("timeStep", 1/60.)
    params.set_param("maxCycles", 800)
    params.set_param("spaceWidth", 10) # cm
    params.set_param("defaultKm", 0.000001)
    params.set_param("maxSpaceBiomass", 1000)

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "I"].values[0] / control.loc[control.cycle == 800, "I"].values[0]
    results.append(proportion)



# now loop through new midpoints until we get close to 50% growth
while all(np.abs(np.array(results) - 0.5) > 0.0001):
    print(f"current kms: {kms}")
    print(f"current proportions: {np.array(results)}")
    start = max([i for i in range(len(results)) if results[i] < 0.5])
    end = min([i for i in range(len(results)) if results[i] > 0.5])
    first_km = kms[start]
    last_km = kms[end]
    new_middle = (first_km + last_km) / 2
    kms = [first_km, new_middle, last_km]
    results = [results[start],
                    0,
                    results[end]]
    km = new_middle
    
    I_comets = c.model(I)
    I_comets.id = "I"
    I_comets.open_exchanges()
    I_comets.initial_pop = [[0,0, 0.01]]
    I_comets.add_multitoxin(rxn_num, exch_inds, bound, vmax, [km], hills)

    models = [I_comets, B_comets]

    layout = c.layout(models)
    for met in unlimited:
        met = met[3:]
        if met in layout.media.metabolite.values:
            layout.set_specific_metabolite(met, 1000.)
    layout.set_specific_metabolite("glc__D_e", 5.)

    sim = c.comets(layout, params)
    sim.COMETS_HOME = "/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/"
    sim.set_classpath("bin", '/home/jeremy/Dropbox/work_related/harcombe_lab/segre/jars/comets_multitoxin.jar')

    sim.working_dir = "../comets_tests/"
    sim.run()
    proportion = sim.total_biomass.loc[sim.total_biomass.cycle == 800, "I"].values[0] / control.loc[control.cycle == 800, "I"].values[0]
    results[1] = proportion
                 


Running COMETS simulation ...
Done!
I_gr_max: 0.9047680854178781

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!

Running COMETS simulation ...
Done!
current kms: [1e-05, 0.0001, 0.001]
current proportions: [0.06779114 0.67909666 0.99887674]

Running COMETS simulation ...
Done!
current kms: [1e-05, 5.5e-05, 0.0001]
current proportions: [0.06779114 0.31198624 0.67909666]

Running COMETS simulation ...
Done!
current kms: [5.5e-05, 7.75e-05, 0.0001]
current proportions: [0.31198624 0.46881435 0.67909666]

Running COMETS simulation ...
Done!
current kms: [7.75e-05, 8.875e-05, 0.0001]
current proportions: [0.46881435 0.5647517  0.67909666]

Running COMETS simulation ...
Done!
current kms: [7.75e-05, 8.3125e-05, 8.875e-05]
current proportions: [0.46881435 0.51505284 0.5647517 ]

Running COMETS simulation ...
Done!
current kms: [7.75e-05, 8.031250000000001e-05, 8.3125e-05]
current proportions: [0.46881435 0.49128304 0.51505284]

Running COMETS simulation ...
Done!
c

In [101]:
print(kms)
print(results)

[8.1279296875e-05, 8.13232421875e-05, 8.13671875e-05]
[0.4995427253875728, 0.49997474416161036, 0.5003344988950044]
